In [48]:
import numpy as np
import pandas as pd
from scipy import misc

import matplotlib.pyplot as plt
%matplotlib inline 

import cv2
import os

In [21]:
# Fixed for our Cats & Dogs classes
NUM_CLASSES = 10

# Fixed for Cats & Dogs color images
CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 10
BATCH_SIZE_VALIDATION = 10

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [22]:
test_path = 'food-102/test'

In [23]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

image_size = IMAGE_RESIZE
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [24]:
model = load_model('my_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


## Testing

In [25]:
test_generator = data_generator.flow_from_directory(
    directory = test_path,
    target_size = (image_size, image_size),
    batch_size = BATCH_SIZE_TESTING,
    class_mode = None,
    shuffle = False,
    seed = 123
)

Found 1 images belonging to 1 classes.


In [50]:
from keras.preprocessing import image
img_path = 'pizza.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

pred = model.predict(x,verbose = 1)


1/1 [==============================] - 1s 505ms/step


In [42]:
# Reset before each call to predict
#test_generator.reset()
#pred = model.predict_generator(test_generator, steps = len(test_generator), verbose = 1)
predicted_class_indices = np.argmax(pred, axis = 1)

In [43]:
predicted_class_indices

array([8])

In [44]:
#pred.argsort(1)
top_preds = pred[0].argsort()[-3:][::-1] # get top 5 predictions
top_preds
#top_pred_names = [data.classes[i] for i in top_preds]

array([8, 4, 3])

In [45]:
train_generator.class_indices.items()

dict_items([('cup_cakes', 0), ('donuts', 1), ('french_fries', 2), ('french_toast', 3), ('fried_rice', 4), ('hamburger', 5), ('hot_and_sour_soup', 6), ('ice_cream', 7), ('pizza', 8), ('samosa', 9)])

In [46]:
predicted = []
for i in range(0,1):
    for j in top_preds:
        for key, value in train_generator.class_indices.items():
            if j == value:
                predicted.append(key)
predicted

['pizza', 'fried_rice', 'french_toast']